In [1]:
import pandas as pd
import numpy as np
import random
import json
import re
import os

from datetime import datetime

# IDENTIFY ALL JSON FILES

In [2]:
files = []

for f in os.listdir():
    if re.findall('.json', f) != []:
        files.append(f)
    else:
        pass

files

['brands.json', 'receipts.json', 'users.json']

# DEFINE FUNCTIONS

In [3]:
def high_lvl_check(df):
    print(f'There are {len(df[df._id.duplicated()])} duplicate records under "_id"')
    print(f'There are {len(df[df._id.isnull()])} records with missing/null values under "_id" field')
    print(f'There are {len(df[df.isnull().any(axis=1)])} records with missing/null values under any field')
    
    multi_dtypes = 0
    
    for c in df.columns:
        
        df_subset = df[~df[c].isnull()].reset_index(drop=True)
        
        dtypes = []
        
        for i in list(range(len(df_subset))):
            dtypes.append(type(df_subset[c][i]))
            
        if len(list(set(dtypes))) > 1:
            multi_dtypes += 1
            print(f'Column "{c}" contains {list(set(dtypes))} dtypes')
            
    if multi_dtypes == 0:
        print('All fields contain consistent data types')

In [35]:
def common_cols(df1, df2):
    
    c1 = df1.columns.tolist()
    c2 = df2.columns.tolist()
    
    return [c for c in c1 if c in c2 and c != '_id']

# ANALYZE brands.json FILE

In [4]:
# READ + VIEW DATASTRUCTURE
df_brands = pd.read_json('brands.json', lines=True)
df_brands.head(3)

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176


In [5]:
high_lvl_check(df_brands)

There are 0 duplicate records under "_id"
There are 0 records with missing/null values under "_id" field
There are 912 records with missing/null values under any field
All fields contain consistent data types


# ANALYZE receipts.json FILE

In [6]:
# READ + VIEW DATASTRUCTURE
df_receipts = pd.read_json('receipts.json', lines=True)
df_receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


In [7]:
high_lvl_check(df_receipts)

There are 0 duplicate records under "_id"
There are 0 records with missing/null values under "_id" field
There are 665 records with missing/null values under any field
All fields contain consistent data types


In [8]:
df_rewardsReceiptItemList = df_receipts[['_id', 'rewardsReceiptItemList']]
df_rewardsReceiptItemList

,_id,rewardsReceiptItemList
0,{'$oid': '5ff1e1eb0a720f0523000575'},"[{'barcode': '4011', 'description': 'ITEM NOT ..."
1,{'$oid': '5ff1e1bb0a720f052300056b'},"[{'barcode': '4011', 'description': 'ITEM NOT ..."
2,{'$oid': '5ff1e1f10a720f052300057a'},"[{'needsFetchReview': False, 'partnerItemId': ..."
3,{'$oid': '5ff1e1ee0a7214ada100056f'},"[{'barcode': '4011', 'description': 'ITEM NOT ..."
4,{'$oid': '5ff1e1d20a7214ada1000561'},"[{'barcode': '4011', 'description': 'ITEM NOT ..."
...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},"[{'barcode': 'B076FJ92M4', 'description': 'mue..."
1115,{'$oid': '603d0b710a720fde1000042a'},NaN
1116,{'$oid': '603cf5290a720fde10000413'},NaN
1117,{'$oid': '603ce7100a7217c72c000405'},"[{'barcode': 'B076FJ92M4', 'description': 'mue..."


In [9]:
df_rewardsReceiptItemList['keys_ct'] = df_rewardsReceiptItemList.rewardsReceiptItemList.apply(lambda i: len(list(i[0].keys())) if isinstance(i, list) else 0)

C:\Users\teddy\Envs\main_env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
print(f'For the receipts.json dataset, under the rewardsReceiptItemList field, some records have a max of {df_rewardsReceiptItemList.keys_ct.max()} nested keys/fields, while some records have {df_rewardsReceiptItemList.keys_ct.min()}. See the distribution below: \n\n {df_rewardsReceiptItemList.keys_ct.value_counts()}')

For the receipts.json dataset, under the rewardsReceiptItemList field, some records have a max of 16 nested keys/fields, while some records have 0. See the distribution below: 

 0     440
8     143
11    132
7      91
6      84
12     80
9      68
13     39
14     20
15     11
16     10
10      1
Name: keys_ct, dtype: int64


In [72]:
pct_with = round(len(df_rewardsReceiptItemList[df_rewardsReceiptItemList.keys_ct>0]) / len(df_rewardsReceiptItemList)*100, 0)
pct_without = round(len(df_rewardsReceiptItemList[df_rewardsReceiptItemList.keys_ct==0]) / len(df_rewardsReceiptItemList)*100, 0)

In [75]:
print(f'For receipts.json dataset, {int(pct_with)}% of receipts have item level detail, {int(pct_without)}% of receipts do not have item level detail')

For receipts.json dataset, 61% of receipts have item level detail, 39% of receipts do not have item level detail


# ANALYZE users .json FILE

In [11]:
# READ + VIEW DATASTRUCTURE
df_users = pd.read_json('users.json', lines=True)
df_users.head(3)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [12]:
high_lvl_check(df_users)

There are 283 duplicate records under "_id"
There are 0 records with missing/null values under "_id" field
There are 131 records with missing/null values under any field
All fields contain consistent data types


# ANALYZE DATA FROM parse-files-to-csv.ipynb OUTPUTS 

In [13]:
os.chdir('outputs')

In [15]:
files = []

for f in os.listdir():
    if re.findall('.csv', f) != []:
        files.append(f)
    else:
        pass
files

['brands.csv', 'receipts.csv', 'receipts_v.csv', 'users.csv']

In [17]:
df_brands = pd.read_csv('brands.csv')

In [18]:
df_receipts = pd.read_csv('receipts.csv')

In [19]:
df_receipts_v = pd.read_csv('receipts_v.csv')

In [20]:
df_users = pd.read_csv('users.csv')

In [40]:
# CHECK MAPPING BETWEEN 'brands' and 'receipts'

In [42]:
common_cols(df_brands, df_receipts)

[]

In [43]:
# CHECK MAPPING BETWEEN 'brands' and 'receipts_v'

In [44]:
common_cols(df_brands, df_receipts_v)

['barcode', 'brandCode']

In [ ]:
# CHECK MAPPING BETWEEN 'brands' and 'users'

In [52]:
common_cols(df_brands, df_users)

[]

In [53]:
# CHECK MAPPING BETWEEN 'df_receipts' and 'df_users'

In [54]:
common_cols(df_receipts, df_users)

[]